*This notebook is part of  course materials for CS 345: Machine Learning Foundations and Practice at Colorado State University.
Original versions were created by Asa Ben-Hur and updated by Ross Beveridge.
The content is availabe [on GitHub](https://github.com/asabenhur/CS345).*

*The text is released under the [CC BY-SA license](https://creativecommons.org/licenses/by-sa/4.0/), and code is released under the [MIT license](https://opensource.org/licenses/MIT).*

<a href="https://colab.research.google.com/github//asabenhur/CS345/blob/master/fall24/notebooks/module01_06_perceptron.ipynb">
  <img align="left" src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

# Predicting Valorant Match Outcomes Using Performance Metrics

By Hallie Gurr and Rose Ordway

In [143]:
import sqlite3
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
import numpy as np

#Helper functions
def removeNanRows(X):
    df = pd.DataFrame(X)
    df = df.dropna()
    return df.values

In [144]:
#Database connection
db_path = "data/valorant.sqlite"
conn = sqlite3.connect(db_path)

# Check existing tables names
tables = pd.read_sql_query(
    "SELECT name FROM sqlite_master WHERE type='table';", conn
)
print(tables)

              name
0          Matches
1            Games
2      Game_Rounds
3  Game_Scoreboard


In [145]:
#Creating and Formatting ML Ready Scoreboard Data
from maps.team_map import TEAM_MAP
from maps.role_map import ROLE_MAP

def getGames(conn, limit=None):
    #Looking at GameID, TeamAbbreviation, PlayerID, Agent, Kills, Deaths, Assists, HS_Percent, Econ
    headers = ["GameID","Winner"]

    selections = ", ".join(headers)

    conditions = " AND ".join([f"{h} IS NOT NULL" for h in headers])

    query = f"""
    SELECT {selections}
    FROM Games
    WHERE {conditions}
    LIMIT {-1 if limit is None else limit};
    """
    games = pd.read_sql_query(query, conn)
    return games

def generateTeamCompList(scoreboard):
    sb = scoreboard.copy()

    sb["Role"] = sb["Agent"].map(ROLE_MAP)

    sb_with_role = sb.dropna(subset=["Role"])

    comp = (
        sb_with_role
        .groupby(["GameID", "TeamAbbreviation", "Role"])
        .size()
        .unstack(fill_value=0)
        .reindex(columns=["Duelist", "Controller", "Initiator", "Sentinel"], fill_value=0)
    )

    return comp

def getScoreboard(conn, limit=None):
    #Looking at GameID, TeamAbbreviation, PlayerID, Agent, Kills, Deaths, Assists, HS_Percent, Econ
    headers = ["GameID","TeamAbbreviation", "PlayerID", "Agent", "Kills", "Deaths", 
           "Assists", "HS_Percent", "Econ"]

    selections = ", ".join(headers)

    conditions = " AND ".join([f"{h} IS NOT NULL" for h in headers])

    query = f"""
    SELECT {selections}
    FROM Game_Scoreboard
    WHERE {conditions}
    LIMIT {-1 if limit is None else limit};
    """
    scoreboard = pd.read_sql_query(query, conn)
    return scoreboard

def assignTeamCompToScoreboard(scoreboard, df_teamcomp):
    df_teamcomp = df_teamcomp.reset_index() 

    merged = scoreboard.merge(
        df_teamcomp,
        on=["GameID", "TeamAbbreviation"],
        how="left"
    )

    return merged

def generateMLReadySB(conn, limit=None):
    scoreboard = getScoreboard(conn, limit=limit)

    df_teamcomp = generateTeamCompList(scoreboard) 

    scoreboard = assignTeamCompToScoreboard(scoreboard, df_teamcomp)

    scoreboard = addWinnerToScoreboard(conn, scoreboard)

    feature_cols = ["Kills", "Deaths", "Assists", "HS_Percent", "Econ",
                    "Duelist", "Controller", "Initiator", "Sentinel", "TeamWin"]
    features = scoreboard[feature_cols].to_numpy()

    X = removeNanRows(features)
    return X

def addWinnerToScoreboard(conn, scoreboard):
    games = getGames(conn)

    game_winner_map = dict(zip(games["GameID"], games["Winner"]))
    scoreboard["WinningAbbrev"] = scoreboard["GameID"].map(game_winner_map).map(TEAM_MAP)

    scoreboard["TeamWin"] = (scoreboard["TeamAbbreviation"] == scoreboard["WinningAbbrev"]).astype(int)

    return scoreboard

In [146]:
dataLimit = None

mlscoreboard = generateMLReadySB(conn, limit=dataLimit)
print(mlscoreboard.shape)

df = pd.DataFrame(mlscoreboard, columns=[
    "Kills", "Deaths", "Assists", "HS_Percent", "Econ","Duelist", "Controller", "Initiator", "Sentinel","TeamWin"])
print(df)

(147702, 10)
        Kills  Deaths  Assists  HS_Percent  Econ  Duelist  Controller  \
0        24.0    10.0      3.0        0.31  74.0      1.0         1.0   
1        16.0    10.0      7.0        0.16  67.0      1.0         1.0   
2        17.0     9.0      8.0        0.27  58.0      1.0         1.0   
3        17.0    12.0      2.0        0.19  48.0      1.0         1.0   
4         5.0    13.0      3.0        0.22  21.0      1.0         1.0   
...       ...     ...      ...         ...   ...      ...         ...   
147697   13.0    12.0      1.0        0.18  61.0      2.0         1.0   
147698    4.0    13.0      4.0        0.33  32.0      2.0         1.0   
147699    4.0    15.0      0.0        0.13  21.0      2.0         1.0   
147700    3.0    14.0      4.0        0.19  29.0      2.0         1.0   
147701    3.0    14.0      2.0        0.12  18.0      2.0         1.0   

        Initiator  Sentinel  TeamWin  
0             2.0       1.0      1.0  
1             2.0       1.0     

In [147]:
# Some of the teams have less than 5 players (due to missing data) percent of missing players: 
# (I might have fixed this but I'm going to leave it for now)
def getMissingPlayerStats(comp):
    scoreboard = getScoreboard(conn, limit=dataLimit)
    comp["RoleCount"] = comp.sum(axis=1)
    unique_ids = scoreboard["GameID"].nunique() *10
    print(f"Actual number of entries: {unique_ids}")
    print(f"Number of entries with role: {comp['RoleCount'].sum()}")
    print(f"Number of missing entries: {unique_ids - comp['RoleCount'].sum()}")
    print(f"Percent missing: {(unique_ids - comp['RoleCount'].sum()) / unique_ids* 100:.2f}%")

getMissingPlayerStats(generateTeamCompList(getScoreboard(conn, limit=dataLimit)))


Actual number of entries: 148480
Number of entries with role: 147702
Number of missing entries: 778
Percent missing: 0.52%


.52% Noise is complete within acceptable ranges in most large ML models. Data featuring null values was removed causing this issue. Some team names were not imputed correctly leading to less agents on a team composition than normal.

In [148]:
tables = conn.execute("SELECT name FROM sqlite_master WHERE type='table';").fetchall()
print("Tables:", tables)

for t in tables:
    name = t[0]
    print(f"\n--- Columns in {name} ---")
    cols = conn.execute(f"PRAGMA table_info({name});").fetchall()
    for c in cols:
        print(c)


Tables: [('Matches',), ('Games',), ('Game_Rounds',), ('Game_Scoreboard',)]

--- Columns in Matches ---
(0, 'MatchID', 'TEXT', 0, None, 0)
(1, 'Date', 'TEXT', 0, None, 0)
(2, 'Patch', 'TEXT', 0, None, 0)
(3, 'EventID', 'TEXT', 0, None, 0)
(4, 'EventName', 'TEXT', 0, None, 0)
(5, 'EventStage', 'TEXT', 0, None, 0)
(6, 'Team1ID', 'INTEGER', 0, None, 0)
(7, 'Team2ID', 'INTEGER', 0, None, 0)
(8, 'Team1', 'TEXT', 0, None, 0)
(9, 'Team2', 'TEXT', 0, None, 0)
(10, 'Team1_MapScore', 'INTEGER', 0, None, 0)
(11, 'Team2_MapScore', 'INTEGER', 0, None, 0)

--- Columns in Games ---
(0, 'GameID', 'TEXT', 0, None, 0)
(1, 'MatchID', 'TEXT', 0, None, 0)
(2, 'Map', 'TEXT', 0, None, 0)
(3, 'Team1ID', 'INTEGER', 0, None, 0)
(4, 'Team2ID', 'INTEGER', 0, None, 0)
(5, 'Team1', 'TEXT', 0, None, 0)
(6, 'Team2', 'TEXT', 0, None, 0)
(7, 'Winner', 'TEXT', 0, None, 0)
(8, 'Team1_TotalRounds', 'INTEGER', 0, None, 0)
(9, 'Team2_TotalRounds', 'INTEGER', 0, None, 0)
(10, 'Team1_SideFirstHalf', 'TEXT', 0, None, 0)
(11, 'T